In [2]:
!pip3 install -r requirements.txt

In [3]:
import osmnx as ox
import matplotlib.pyplot as plt
import osmium as osm
import pandas as pd

/Users/fabian/Documents/GitHub/osm-times-of-crisis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []

    def tag_inventory(self, elem, elem_type):
        for tag in elem.tags:
            self.osm_data.append([elem_type, 
                                   elem.id, 
                                   elem.version,
                                   elem.visible,
                                   pd.Timestamp(elem.timestamp),
                                   elem.uid,
                                   elem.user,
                                   elem.changeset,
                                   len(elem.tags),
                                   tag.k, 
                                   tag.v])

    def node(self, n):
        self.tag_inventory(n, "node")

    def way(self, w):
        self.tag_inventory(w, "way")

    def relation(self, r):
        self.tag_inventory(r, "relation")


: 

Get all the footways

In [5]:
#map_file = 'changesets-240909.osm'
map_file = '../DataExtracts/osmium-london-extract.osm'

highways = ox.features.features_from_xml(map_file, tags={'highway':True})
highways.plot(figsize=(40,40))

display(highways)


In [ ]:
#map_file = 'changesets-240909.osm'
map_file = '../DataExtracts/broxbourne-roads-2024-01.osm'
map_file = '../DataExtracts/uk-roads.osm'

highways = ox.features.features_from_xml(map_file, tags={'highway':True})
highways.plot(figsize=(40,40))

display(highways)

Another way to get all the footways from the input dataset, using OSMHandler rather than osmnx

In [18]:
osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file(map_file)

# transform the list into a pandas DataFrame
data_colnames = ['type', 'id', 'version', 'visible', 'ts', 'uid',
                 'user', 'chgset', 'ntags', 'tagkey', 'tagvalue']
df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)

# df.loc[(df['col1'] == value) & (df['col2'] < value)]
footways_osm_handler = df_osm.loc[(df_osm['tagkey'] == 'highway') & (df_osm['tagvalue'] == 'footway')]
display(footways_osm_handler)

,type,id,version,visible,ts,uid,user,chgset,ntags,tagkey,tagvalue
34926,way,2414995,9,True,2022-05-01 01:52:19+00:00,7096975,Peter Newman,120403713,4,highway,footway
36152,way,3630107,17,True,2022-06-19 16:18:23+00:00,338535,ramthelinefeed,122585552,6,highway,footway
36392,way,3690070,2,True,2011-07-10 14:45:29+00:00,3980,TomH,8684315,1,highway,footway
36393,way,3690071,2,True,2011-07-10 14:45:29+00:00,3980,TomH,8684315,1,highway,footway
36400,way,3699510,1,True,2006-10-16 17:54:27+00:00,3980,TomH,133526,2,highway,footway
...,...,...,...,...,...,...,...,...,...,...,...
133075,way,1233542547,1,True,2023-12-23 15:47:30+00:00,3951648,rskedgell,145441512,4,highway,footway
133139,way,1233592491,1,True,2023-12-23 19:00:05+00:00,3951648,rskedgell,145447082,6,highway,footway
133235,way,1234716211,1,True,2023-12-27 20:30:27+00:00,13341565,Falsernet,145591619,7,highway,footway
133270,way,1234874651,1,True,2023-12-28 11:47:16+00:00,3951648,rskedgell,145615658,4,highway,footway


To get the Unique Changesets in a downloaded dataset

In [19]:

unique_changesets = df_osm['chgset'].unique()
print(sorted(unique_changesets))

[8164, 11803, 12203, 13186, 13686, 14075, 14480, 17305, 17972, 21112, 25214, 32287, 32641, 36788, 38283, 39033, 39996, 40539, 40790, 40833, 40895, 41403, 41846, 42248, 43216, 43504, 43700, 47664, 62973, 63419, 63744, 71528, 82552, 83486, 88509, 88588, 88986, 90231, 91154, 91564, 93352, 98013, 100678, 104952, 108596, 109359, 109602, 113978, 114256, 119769, 123661, 124275, 124877, 126009, 129388, 129752, 130541, 133526, 133583, 136149, 137889, 144443, 154654, 154852, 155348, 156370, 157573, 158422, 159480, 163206, 168095, 191422, 193511, 194103, 201373, 201708, 201738, 203135, 209280, 209547, 212351, 224351, 228021, 236236, 236850, 236944, 238268, 241514, 242868, 244502, 244995, 249827, 249977, 250023, 250717, 252329, 252439, 252514, 252741, 253999, 258855, 261363, 264330, 264779, 266820, 268472, 268902, 271814, 274807, 276724, 281038, 287965, 291268, 294434, 297693, 302576, 302993, 303402, 309129, 311752, 315791, 319990, 320657, 320664, 322859, 323528, 329016, 330083, 335863, 340956, 34

For each Footway that we have, match with the changeset